# Transformata Fouriera dla obrazów cyfrowych. Filtracja w dziedzinie częstotliwości.

## Cel ćwiczenia

- Zapoznanie z wykorzystaniem transformaty Fouriera w przetwarzaniu obrazów cyfrowych.
- Zapoznanie z pojęciem F-obrazu (amplitudy i fazy).
- Zapoznanie z własnościami transformaty Fouriera.
- Zapoznanie z filtracją w dziedzinie częstotliwości.

Na jednym z poprzednich ćwiczeń zetknęliśmy się z pojęciem konwolucji.
Przykładem może być filtracja dolno i górnoprzepustowa.
Operacja ta odpowiada mnożeniu w dziedzinie częstotliwości zgodnie z zależnością:

\begin{equation}
\mathcal{F}(g(x,y)*h(x,y)) = \mathcal{F}(g(x,y)) \cdot \mathcal{F}(h(x,y))
\tag{1}
\end{equation}

gdzie: $\mathcal{F}$ oznacza transformatę Fouriera, a $*$ jest splotem.

Operacja filtracji w dziedzinie częstotliwości może okazać się bardziej efektywna, jeżeli operacje $fft$ i $ifft$ (odpowiednio szybka transformata Fouriera -- *fast Fourier transform* -- oraz odwrotna szybka transformata Fouriera -- *inverse fast Fourier transform*) zajmą mniej czasu niż klasyczna konwolucja (zazwyczaj ma to miejsce dla dużego obrazu, z dużą maską).

Sama filtracja w dziedzinie częstotliwości to mnożenie punktowe całego obrazu przez jedną maskę.

W przypadku filtracji w dziedzinie częstotliwości zakłada się, że obraz "zawija się" na brzegach, co może powodować pewne artefakty (zostanie to pokazane w trakcie ćwiczenia).

W dziedzinie częstotliwości "działają" tylko filtry liniowe.
Filtry medianowe, maksymalne, minimalne itp. nie mają swoich odpowiedników.

## Dwuwymiarowa transformata Fouriera

1. Wczytaj plik "dwieFale.bmp" w skali szarości.
Jest to obraz powstały na podstawie następującej zależności:

\begin{equation}
L(m, n) = 128 + 127 \cdot \cos(\frac{2\pi m}{32}+\frac{3\pi}{4}) \cdot \cos(\frac{2\pi n}{8}-\frac{\pi}{2})
\tag{2}
\end{equation}

gdzie: $m$ i $n$ są odpowiednio numerami wierszy i kolumn.

2. Do realizacji dwuwymiarowej transformaty Fouriera służy funkcja `cv2.dft`.
Ustaw flagę `flags=cv2.DFT_COMPLEX_OUTPUT`.
Wykonaj transformatę na wczytanym obrazie.
W ten sposób uzyskuje się tzw. F-obraz.

3. Najniższe częstotliwości znajdują się w lewym-górnym rogu obrazu.
Dla celów wizualizacji (ale też przetwarzania) często wykonuje się tzw. przesunięcie F-obrazu, które powoduje, że niskie częstotliwości przesuwane są do środka obrazu.
Wykorzystaj funkcję `np.fft.fftshift`.
Jako pierwszy argument podaj wynik transformaty Fouriera.
Jako drugi argument podaj numery osi, wzdłuż których należy wykonać operację.
Pierwsza oś odnosi się do wierszy obrazu.
Druga oś odnosi się do kolumn obrazu.
Trzecia oś to część rzeczywista (`[:, :, 0]`) lub urojona (`[:, :, 1]`).
W naszym przypadku argument powinien wyglądać tak `[0,1]`.

4. Wyświetl wynik transformaty.
Na wspólnym wykresie umieść obraz oryginalny, amplitudę i fazę F-obrazu.
Amplitudę i fazę wyznacz za pomocą funkcji `cv2.cartToPolar`.
Pierwszym argumentem funkcji jest część rzeczywista wyniku, a drugim urojona.
Uwaga. W razie wątpliwości proszę sprawdzić rozmiary rezultatu transformaty Fouriera oraz przesunięcia.

5. Dla wizualizacji oblicz logarytm dziesiętny amplitudy: `ALog = np.log10(A + 1)`.
Wyświetl go zamiast amplitudy na poprzednim wykresie.

6. Wczytaj obrazy *kolo.bmp*, *kwadrat.bmp*, *kwadrat45.bmp*, *trojkat.bmp*.
Czy analizując F-obraz można coś powiedzieć o kierunku krawędzi obiektów?

7. Sprawdź (empirycznie) poprawność stwierdzenia:

`Dwuwymiarowa transformata Fouriera jest złożeniem dwóch transformat jednowymiarowych (wykonanych np. najpierw wierszowo, a później kolumnowo).` 
Jednowymiarowa transformata realizowana jest za pomocą funkcji fft (z bibloteki Numpy).

Wykonaj najpierw transformatę po wierszach: `FRow = np.fft.fft(I, axis=0)`.
Następnie po kolumnach: `FCol = np.fft.fft(FRow, axis=1)`.
Numpy zwraca wynik jako tablicę liczb zespolonych.
Część rzeczywistą można otrzymać w następujący sposób: `FCol.real`, a urojoną: `FCol.imag`.
Porównaj tak uzyskany wynik z rezultatem działania funkcji `cv2.dft`.
Można to zrobić wizualnie lub z wykorzystaniem funkcji `cv2.absdiff`.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os
from scipy import ndimage

if not os.path.exists("dwieFale.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/dwieFale.bmp --no-check-certificate
if not os.path.exists("kolo.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kolo.bmp --no-check-certificate
if not os.path.exists("kwadrat.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadrat.bmp --no-check-certificate
if not os.path.exists("kwadrat45.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadrat45.bmp --no-check-certificate
if not os.path.exists("kwadratKL.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadratKL.bmp --no-check-certificate
if not os.path.exists("kwadratS.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadratS.bmp --no-check-certificate
if not os.path.exists("kwadratT.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/kwadratT.bmp --no-check-certificate
if not os.path.exists("lena.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/lena.bmp --no-check-certificate
if not os.path.exists("trojkat.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/trojkat.bmp --no-check-certificate
if not os.path.exists("literki.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/literki.bmp --no-check-certificate
if not os.path.exists("wzorA.bmp") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/08_Fourier/wzorA.bmp --no-check-certificate

I_Fale = cv2.imread('dwieFale.bmp', cv2.IMREAD_GRAYSCALE)
I_kolo = cv2.imread('kolo.bmp', cv2.IMREAD_GRAYSCALE)
I_kwadrat = cv2.imread('kwadrat.bmp', cv2.IMREAD_GRAYSCALE)
I_kwadrat45 = cv2.imread('kwadrat45.bmp', cv2.IMREAD_GRAYSCALE)
I_kwadratKL = cv2.imread('kwadratKL.bmp', cv2.IMREAD_GRAYSCALE)
I_kwadratS = cv2.imread('kwadratS.bmp', cv2.IMREAD_GRAYSCALE)
I_kwadratT = cv2.imread('kwadratT.bmp', cv2.IMREAD_GRAYSCALE)
I_lena = cv2.imread('lena.bmp', cv2.IMREAD_GRAYSCALE)
I_trojkat = cv2.imread('trojkat.bmp', cv2.IMREAD_GRAYSCALE)

images = {
    "dwieFale": I_Fale,
    "kolo": I_kolo,
    "kwadrat": I_kwadrat,
    "kwadrat45": I_kwadrat45,
    "kwadratKL": I_kwadratKL,
    "kwadratS": I_kwadratS,
    "kwadratT": I_kwadratT,
    "lena": I_lena,
    "trojkat": I_trojkat
}

num_images = len(images)
cols = 3
rows = (num_images + cols - 1) // cols

fig, axs = plt.subplots(rows, cols, figsize=(12, 12))

for i, (name, image) in enumerate(images.items()):
    row = i // cols
    col = i % cols
    axs[row, col].imshow(image, 'gray')
    axs[row, col].set_title(name)
    axs[row, col].axis('off')


for i in range(num_images, rows * cols):
    row = i // cols
    col = i % cols
    axs[row, col].axis('off')

plt.tight_layout()
plt.show()

In [ ]:
def fourier1(image):
    image = image.astype('float32')
    F_fcn = cv2.dft(image, flags = cv2.DFT_COMPLEX_OUTPUT)
    F_fcn = np.fft.fftshift(F_fcn, [0, 1])
    magnitude, phase = cv2.cartToPolar(F_fcn[:, :, 0], F_fcn[:, :, 1])
    return F_fcn, magnitude, phase

def fourier2(image):
    image = image.astype('float32')
    F_fcn = cv2.dft(image, flags = cv2.DFT_COMPLEX_OUTPUT)
    F_fcn = np.fft.fftshift(F_fcn, [0, 1])
    magnitude, phase = cv2.cartToPolar(F_fcn[:, :, 0], F_fcn[:, :, 1])
    magnitude_log = np.log10(magnitude + 1)
    return F_fcn, magnitude_log, phase

def fourier3(image):
    FRow = np.fft.fft(image, axis = 0)
    F_fcn = np.fft.fft(FRow, axis = 1)
    magnitude, phase = cv2.cartToPolar(F_fcn.real, F_fcn.imag)
    magnitude_log = np.log10(magnitude + 1)
    return F_fcn, magnitude_log, phase

def plot_fourier_transform(original, magnitude, phase, suptitle):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle('Image: ' + suptitle)

    axs[0].imshow(original, cmap='gray')
    axs[0].set_title('Original Image')
    axs[0].axis('off')

    axs[1].imshow(magnitude, cmap='gray')
    axs[1].set_title('Magnitude')
    axs[1].axis('off')

    axs[2].imshow(phase, cmap='gray')
    axs[2].set_title('Phase')
    axs[2].axis('off')

    plt.show()

def plot_fourier_transform_log(original, magnitude, phase, suptitle):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle('Image: ' + suptitle)

    axs[0].imshow(original, cmap='gray')
    axs[0].set_title('Original Image')
    axs[0].axis('off')

    axs[1].imshow(magnitude, cmap='gray')
    axs[1].set_title('Log Magnitude')
    axs[1].axis('off')

    axs[2].imshow(phase, cmap='gray')
    axs[2].set_title('Phase')
    axs[2].axis('off')

    plt.show()

def plot_comparision2(original1, magnitude1, phase1, original2, magnitude2, phase2, suptitle):
    
    fig, axs = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle('Image: ' + suptitle)

    axs[0, 0].imshow(original1, cmap='gray')
    axs[0, 0].set_title('Original Image')
    axs[0, 0].axis('off')

    axs[0, 1].imshow(magnitude1, cmap='gray')
    axs[0, 1].set_title('Log Magnitude')
    axs[0, 1].axis('off')

    axs[0, 2].imshow(phase1, cmap='gray')
    axs[0, 2].set_title('Phase')
    axs[0, 2].axis('off')

    axs[1, 0].imshow(original2, cmap='gray')
    axs[1, 0].set_title('Original Image')
    axs[1, 0].axis('off')

    axs[1, 1].imshow(magnitude2, cmap='gray')
    axs[1, 1].set_title('Log Magnitude')
    axs[1, 1].axis('off')

    axs[1, 2].imshow(phase2, cmap='gray')
    axs[1, 2].set_title('Phase')
    axs[1, 2].axis('off')
    plt.show()

def plot_fourier2_comparision3(original1, original2, original3, suptitle):    
    _, magnitude1, phase1 = fourier2(original1)
    _, magnitude2, phase2 = fourier2(original2)
    _, magnitude3, phase3 = fourier2(original3)

    fig, axs = plt.subplots(3, 3, figsize=(15, 10))
    fig.suptitle('Image: ' + suptitle)

    axs[0, 0].imshow(original1, cmap='gray')
    axs[0, 0].set_title('Original Image')
    axs[0, 0].axis('off')

    axs[0, 1].imshow(magnitude1, cmap='gray')
    axs[0, 1].set_title('Log Magnitude')
    axs[0, 1].axis('off')

    axs[0, 2].imshow(phase1, cmap='gray')
    axs[0, 2].set_title('Phase')
    axs[0, 2].axis('off')

    axs[1, 0].imshow(original2, cmap='gray')
    axs[1, 0].set_title('Original Image')
    axs[1, 0].axis('off')

    axs[1, 1].imshow(magnitude2, cmap='gray')
    axs[1, 1].set_title('Log Magnitude')
    axs[1, 1].axis('off')

    axs[1, 2].imshow(phase2, cmap='gray')
    axs[1, 2].set_title('Phase')
    axs[1, 2].axis('off')

    axs[2, 0].imshow(original3, cmap='gray')
    axs[2, 0].set_title('Original Image')
    axs[2, 0].axis('off')

    axs[2, 1].imshow(magnitude3, cmap='gray')
    axs[2, 1].set_title('Log Magnitude')
    axs[2, 1].axis('off')

    axs[2, 2].imshow(phase3, cmap='gray')
    axs[2, 2].set_title('Phase')
    axs[2, 2].axis('off')
    plt.show()



In [ ]:
for name, image in images.items():
    _, magnitude_log, phase = fourier1(image)
    plot_fourier_transform(image, magnitude_log, phase, name)
    print('\n\n\n\n\n')

In [ ]:
for name, image in images.items():
    _, magnitude_log, phase = fourier2(image)
    plot_fourier_transform_log(image, magnitude_log, phase, name)
    print('\n\n\n\n\n')

In [ ]:
for name, image in images.items():
    _, magnitude_log, phase = fourier3(image)
    plot_fourier_transform_log(image, magnitude_log, phase, name)
    print('\n\n\n\n\n')

In [ ]:
for name, image in images.items():
    _, magnitude_log1, phase1 = fourier2(image)
    _, magnitude_log2, phase2 = fourier3(image)
    plot_comparision2(image, magnitude_log1, phase1, image, magnitude_log2, phase2, name)
    print('\n\n\n\n\n')

## Własności dwuwymiarowej transformaty Fouriera

1. Zbadaj jak zmienia się F-obraz (amplituda i faza) podczas następujących operacji: translacja, rotacja, zmiana rozmiaru, kombinacja liniowa.
Wykorzystaj stworzony wcześniej kod.<br>
Uwaga. Należy użyć przygotowanych obrazów, a nie "generować" własne.
2. Do badania translacji wykorzystaj obrazy *kwadrat.bmp* i *kwadratT.bmp*.
3. Przy badaniu rotacji wykorzystaj obrazy *kwadrat.bmp* i *kwadrat45.bmp*.
4. Przy badaniu zmiany rozmiaru wykorzystaj obrazy *kwadrat.bmp* i *kwadratS.bmp*.
5. Przy badaniu kombinacji liniowej wykorzystaj obrazy *kwadrat.bmp*, *kwadrat45.bmp* i *kwadratKL.bmp*.

In [ ]:
# Translacja
_, magnitude_log1, phase1 = fourier2(I_kwadrat)
_, magnitude_log2, phase2 = fourier2(I_kwadratT)
plot_comparision2(I_kwadrat, magnitude_log1, phase1, I_kwadratT, magnitude_log2, phase2, "kwadrat and kwadratT")

In [ ]:
# Rotacja
_, magnitude_log1, phase1 = fourier2(I_kwadrat)
_, magnitude_log2, phase2 = fourier2(I_kwadrat45)
plot_comparision2(I_kwadrat, magnitude_log1, phase1, I_kwadrat45, magnitude_log2, phase2, "kwadrat and kwadrat45")

In [ ]:
# Zmiana rozmiaru
_, magnitude_log1, phase1 = fourier2(I_kwadrat)
_, magnitude_log2, phase2 = fourier2(I_kwadratS)
plot_comparision2(I_kwadrat, magnitude_log1, phase1, I_kwadratS, magnitude_log2, phase2, "kwadrat and kwadratS")

In [ ]:
plot_fourier2_comparision3(I_kwadrat, I_kwadrat45, I_kwadratKL, "kwadrat, kwadrat45 and kwadratKL")

## Odwrotna dwuwymiarowa transformata Fouriera

1. Wykorzystaj stworzony wcześniej kod. Wybierz dowolny obraz np "kolo.bmp".
2. Przed realizacją odwrotnego przekszałcenia należy wykonać odwrotne przesunięcie.
Wykorzystaj funkcję `np.fft.ifftshift`.
Pierwszym argumentem jest wynik transformaty Fouriera.
Drugim argumentem są numery osi, wzdłuż których należy wykonać operację.
3. Wykonaj odwrotną transformatę Fouriera za pomocą funkcji `cv2.idft`.
Jako drugi argument przekaż następujące flagi: `flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT`.
Wynik może mieć małą część urojoną przez błędy numeryczne.
Aby pozbyć się tego efekty należy obliczyć amplitudę:
        `imgIFFT = cv2.magnitude(ifft[:, :, 0], ifft[:, :, 1])`
Następnie wynik należy zaokrąglić (`np.round`) i zrzutować do typu `uint8`.
4. Wyświetl wynik.
Sprawdź (wizualnie i poprzez odjęcie) czy obraz oryginalny i po przekształceniach są takie same.

In [ ]:
def inverse_fourier_transform(image):
    fft_shifted = np.fft.ifftshift(image, [0, 1])
    ifft = cv2.idft(fft_shifted, flags=cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT)
    imgIFFT = cv2.magnitude(ifft[:, :, 0], ifft[:, :, 1])
    reconstructed_image = np.round(imgIFFT).astype(np.uint8)
    return reconstructed_image

def plot_3(image1, image2, image3, suptitle):
    fig, (ax1,ax2,ax3) = plt.subplots(1, 3,figsize=(20,6))
    fig.suptitle("image: " + suptitle)
    ax1.imshow(image1, 'gray')
    ax1.set_title('Original')
    ax1.axis('off')

    ax2.imshow(image2, 'gray')
    ax2.set_title('IFFT')
    ax2.axis('off')
    
    ax3.imshow(image3, 'gray')
    ax3.set_title('After subtraction')
    ax3.axis('off')
    plt.show()

In [ ]:
for name, image in images.items():
    F_fcn, magnitude_log, phase = fourier2(image)
    inv = inverse_fourier_transform(F_fcn)
    difference = np.abs(image - inv)
    plot_fourier_transform_log(image, magnitude_log, phase, name)
    plot_3(image, inv, difference, name)
    print('\n\n\n\n\n')
    

## Filtracja obrazu w dziedzinie częstotliwości

1. Wczytaj obraz "lena.bmp" w skali szarości.
Wykonaj transformatę Fouriera.
Wykorzystaj stworzony poprzednio kod.
Wyświetl obraz oryginalny, amplitudę (w skali logarytmicznej) i fazę.

2. Przeprowadź filtrację dolnoprzepustową - usuń górne częstotliwości.
Dla F-obrazu po operacji przesunięcia (`fftshift`) niskie częstotliwości leżą w jego centrum.

3. Na początku stwórz filtr "kołowy", dolnoprzepustowy.
Należy wygenerować macierze opisujące przestrzeń w dziedzinie częstotliwości.
Ich rozmiar musi być taki sam jak rozmiar przetwarzanego obrazu.

        lenaSize = I_Lena.shape
        FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(lenaSize[0]))
        FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, lenaSize[1]]))
        FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(lenaSize[1]))
        FSpaceColsM = np.outer(np.ones([1, lenaSize[0]]), FSpaceCols)
        
Powyższy kod wygeneruje dwie znormalizowane macierze częstotliwości: *FSpaceRowsM* i *FSpaceColsM*.
Następnie należy wyznaczyć macierz zawierającą "odległość" od składowej stałej.
        `FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))`

Uwagi:
- funkcja `fftfreq` generuje wektor częstotliwości $[-0.5, 0.5]$ o określonym rozmiarze, przy czym układ wartości jest taki, że najpierw od 0 do 0.5, a potem od -0.5 do 0,
- operacja `fftshift` zmienia ten układ na $[-0.5, 0.5]$,
- mnożenie przez 2 ustala ostatecznie zakres na $[-1, 1]$,
- operacja `outer` to tzw. iloczyn zewnętrzy dwóch wektorów, w naszym przypadku powoduje, że wektor pionowy lub poziomy jest "powielany" odpowiednią liczbę razy.   
- sugeruje się, aby przyglądnąć się jak wygląda macierz `FreqR` - czy to w debugerze, czy poprzez wizualizację.

4. Teraz należy wybrać interesujący zakres.
Tu można zdefiniować typ filtru (dolno, górno, pasmowoprzepustowy).

        FilterF = FreqR <= 0.1 

Filtr należy zwizualizować:

        figFilter = plt.figure()
        axsFilter = figFilter.add_subplot(projection='3d')
        axsFilter.plot_surface(FSpaceRowsM, FSpaceColsM, FilterF, rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)
        figFilter.show()

4. Wykonaj właściwą filtrację, czyli mnożenie F-obrazu przez filtr FilterF.
Trzeba pamiętać, że F-obraz ma 2 kanały (rzeczywisty i urojony).
By mnożenie było możliwe należy więc powielić filtr również na 2 kanały.

        FilterF3 = np.repeat(FilterF[:, :, np.newaxis], 2, axis=2)

5. Wykonaj operację odwrotnego przesunięcia i odwrotnej transformaty.
Oblicz wartość bezwzględną wyniku.
Wykorzystaj funkcję `cv2.magnitude`.
Pierwszym argumentem jest część rzeczywista.
Drugim argumentem jest część urojona.
Wynik wyświetl.

6. Poeksperymentuj z rozmiarem filtru (promieniem).
Zaimplementuj filtr górnoprzepustowy (zmiana znaku przy warunku na odległość) oraz pasmowoprzepustowy (dwa warunki na promień połączone operatorem AND '&' ).
Wykonaj co najmniej trzy filtry i wyświetl wyniki.

7. W ten sposób zaimplementowana filtracja wprowadza pewne artefakty w postaci "pierścieni" wokół krawędzi.
Zapobiec temu zjawisku można poprzez odpowiednie "modelowanie" filtra.
W tym celu wykorzystać należy okna, np. Hamminga, Hanninga, Chebysheva (znane z przetwarzania sygnałów 1D).
Zagadnienie to jest tematem zadania domowego do tego ćwiczenia.

In [ ]:
def filter(image, low_pass: bool, high_pass: bool):
    if low_pass == False and high_pass == False:
        raise ValueError("Wrong input data!!!")
    else:
        F_fcn, magnitude, phase = fourier1(image)
        
        lenaSize = image.shape
        
        FSpaceRows = 2 * np.fft.fftshift(np.fft.fftfreq(lenaSize[0]))
        FSpaceRowsM = np.outer(FSpaceRows, np.ones([1, lenaSize[1]]))
        FSpaceCols = 2 * np.fft.fftshift(np.fft.fftfreq(lenaSize[1]))
        FSpaceColsM = np.outer(np.ones([1, lenaSize[0]]), FSpaceCols)
        
        FreqR = np.sqrt(np.square(FSpaceRowsM) + np.square(FSpaceColsM))

        if low_pass == True and high_pass == False:
            FilterF = FreqR <= 0.1
        elif low_pass == False and high_pass == True:
            FilterF = FreqR >= 0.3
        else:
            low_cutoff = FreqR >= 0.2
            high_cutoff = FreqR <= 0.3
            FilterF = np.logical_and(low_cutoff, high_cutoff)
        
        FilterF3 = np.repeat(FilterF[:, :, np.newaxis], 2, axis=2)
        
        filtered_fft = FilterF3 * F_fcn
        
        ifft = inverse_fourier_transform(filtered_fft)
        
        magnitude_filtered = np.abs(ifft)
        
        difference = np.abs(image - ifft)
        
        plot_fourier_transform(image, magnitude, phase, "Original Image")
        
        figFilter = plt.figure()
        axsFilter = figFilter.add_subplot(projection='3d')
        axsFilter.plot_surface(FSpaceRowsM, FSpaceColsM, FilterF, rstride=1, cstride=1, cmap=plt.get_cmap('gray'), linewidth=0)
        if low_pass == True and high_pass == False:
            plt.title('Low-pass Filter')
        elif low_pass == False and high_pass == True:
            plt.title('High-pass Filter')
        else:
            plt.title('Band-pass Filter')
        figFilter.show()
        
        if low_pass == True and high_pass == False:
            plot_3(image, ifft, difference, "Filtered image (Low-pass)")
        elif low_pass == False and high_pass == True:
            plot_3(image, ifft, difference, "Filtered image (High-pass)")
        else:
            plot_3(image, ifft, difference, "Filtered image (Band-pass)")

In [ ]:
filter(I_lena, low_pass=True, high_pass=False)

In [ ]:
filter(I_lena, low_pass=False, high_pass=True)

In [ ]:
filter(I_lena, low_pass=True, high_pass=True)

## Implementacja wyszukiwania wzorca za pomocą FFT

1. Wczytaj w skali szarości i wyświetl obrazy *literki.bmp* i *wzorA.bmp*.

2. Wyznacz transformatę Fouriera obrazu *literki.bmp*.

3. Obróć drugi obraz o $180^\circ$.
Zastosuj funkcję `np.rot90`.
Pierwszym argumentem jest obracana macierz, a drugim liczba obrotów o $90^\circ$.

4. Należy wyznaczyć transformatę Fouriera obróconego obrazu w taki sposób, żeby miała ona taki sam rozmiar jak pierwszy obraz.
W tym celu należy zastosować *Zero Padding*.
Operacja ta polega na uzupełnieniu obrazu zerami do oczekiwanego rozmiaru.
Uzupełnij obraz zerami z **prawej** strony i z **dołu**.
W tym celu należy wykorzystać funkcję `cv2.copyMakeBorder`.
    - Pierwszym argumentem jest obraz wejściowy.
    - Drugim argumentem jest liczba wierszy u góry.
    - Trzecim argumentem jest liczba wierszy u dołu.
    - Czwartym argumentem jest liczba kolumn z lewej.
    - Piątym argumentem jest liczba kolumn z prawej.
    - Szóstym argumentem jest flaga typu wypełnienia.
    Dla stałej wartości podaj `cv2.BORDER_CONSTANT`.
    - Siódmym argumentem jest wartość pikseli w ramce.
    Przekaż `value=0`.

5. Wyznacz transformatę Fouriera obrazu stworzonego w poprzednim punkcie.

6. Wyniki obu transformat należy przekonwertować do liczb zespolonych.
Obecnie jest to dwukanałowa macierz.
Pierwszy kanał odpowiada za część rzeczywistą.
Drugi kanał odpowiada za część urojoną.
Aby to osiągnąć wystarczy wykonać działanie:

        Complex = Real + Imag * 1j

7. Przemnóż ze sobą zespolone wyniki transformat.

8. Wynik należy powrotnie przekształcić do dwukanałowej macierzy.
Aby to zrobić wykonaj operację:

        CompMat = cv2.merge([np.real(Complex), np.imag(Complex)])

9. Wykonaj odwrotną transformatę Fouriera.
Dodaj flagę `flags=cv2.DFT_COMPLEX_INPUT`.

10. Oblicz wartość bezwzględną wyniku.

11. Wykonaj morfologiczną operację **Top-Hat**, by znaleźć maksima lokalne.
Operacja ta zostanie dokładnej wyjaśniona w jednym z kolejnych ćwiczeń.
W tym celu wykorzystaj operację:

        cv2.morphologyEx(correlation, cv2.MORPH_TOPHAT, np.ones((3, 3), np.uint8))

12. Wyświetl obok siebie obraz wejściowy i wynik wykonanych operacji.
Czy możesz wskazać położenie wzoru na podstawie drugiego obrazu?

In [ ]:
literki = cv2.imread("literki.bmp", cv2.IMREAD_GRAYSCALE)
wzorA = cv2.imread("wzorA.bmp", cv2.IMREAD_GRAYSCALE)

F_fcn1, magnitude_log1, phasel = fourier2(literki)
wzorA_180 = np.rot90(wzorA, 10)

wzorA_padd = cv2.copyMakeBorder(wzorA_180, 234,0,233,0,cv2.BORDER_CONSTANT, value = 0)

F_fcn2, mag_w, phase_w = fourier2(wzorA_padd)

complex_l = F_fcn1[:,:,0] + F_fcn1[:,:,1]*1j
complex_w = F_fcn2[:,:,0] + F_fcn2[:,:,1]*1j

multi = complex_l * complex_w
CompMat = cv2.merge([np.real(multi), np.imag(multi)])

CompMat_s = np.fft.ifftshift(CompMat, [0,1])
f_f = cv2.idft(CompMat_s, flags = cv2.DFT_SCALE | cv2.DFT_COMPLEX_OUTPUT  | cv2.DFT_COMPLEX_INPUT)
img_f_f = cv2.magnitude(f_f[:,:,0], f_f[:,:,1])

CompMat_rev = np.abs(img_f_f)
CompMat_top_hat = cv2.morphologyEx(CompMat_rev, cv2.MORPH_TOPHAT, np.ones((3,3), np.uint8))

plt.figure(figsize=(15,15))

plt.subplot(1,2,1)
plt.imshow(literki,'gray')
plt.title('Original image')
plt.axis('off')

plt.subplot(1,2,2)
plt.imshow(CompMat_top_hat,'gray')
plt.title('Top-Hat transform')
plt.axis('off')

